In [1]:
import streamlit as st
import pandas as pd
import pickle
import smtplib
import random
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from typing import List

# Rutas de arcimport streamlit as st
import pandas as pd
import pickle
import smtplib
import random
import os
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from typing import List

# Rutas de archivos
DATA_FILE = 'birthdays.pkl'
MESSAGES_FILE = 'messages.pkl'
CSV_BIRTHDAYS = 'birthdays.csv'
CSV_MESSAGES = 'messages.csv'

class Person:
    """
    Representa una persona con nombre, fecha de nacimiento y correo electrónico.
    """
    def __init__(self, name: str, birthdate: str, email: str):
        self.name = name
        self.birthdate = datetime.strptime(birthdate, "%Y-%m-%d")
        self.email = email

    def days_until_birthday(self) -> int:
        """Calcula cuántos días faltan para el próximo cumpleaños."""
        today = datetime.today()
        next_birthday = self.birthdate.replace(year=today.year)
        if next_birthday < today:
            next_birthday = next_birthday.replace(year=today.year + 1)
        return (next_birthday - today).days

class BirthdayManager:
    """
    Maneja la lógica del sistema de recordatorio de cumpleaños.
    """
    def __init__(self):
        self.people: List[Person] = self.load_data()
        self.messages: List[str] = self.load_messages()

    def load_data(self) -> List[Person]:
        try:
            with open(DATA_FILE, 'rb') as f:
                return pickle.load(f)
        except (FileNotFoundError, EOFError):
            return []

    def save_data(self):
        with open(DATA_FILE, 'wb') as f:
            pickle.dump(self.people, f)

    def export_birthdays_to_csv(self):
        data = [(p.name, p.birthdate.strftime("%Y-%m-%d"), p.email) for p in self.people]
        df = pd.DataFrame(data, columns=["Nombre", "Cumpleaños", "Correo Electrónico"])
        df.to_csv(CSV_BIRTHDAYS, index=False)

    def add_person(self, name: str, birthdate: str, email: str):
        self.people.append(Person(name, birthdate, email))
        self.save_data()
        self.export_birthdays_to_csv()

    def upcoming_birthdays(self):
        return sorted(self.people, key=lambda p: p.days_until_birthday())

    def load_messages(self) -> List[str]:
        try:
            with open(MESSAGES_FILE, 'rb') as f:
                return pickle.load(f)
        except (FileNotFoundError, EOFError):
            return []

    def save_messages(self):
        with open(MESSAGES_FILE, 'wb') as f:
            pickle.dump(self.messages, f)

    def export_messages_to_csv(self):
        df = pd.DataFrame(self.messages, columns=["Mensaje"])
        df.to_csv(CSV_MESSAGES, index=False)

    def add_message(self, message: str):
        self.messages.append(message)
        self.save_messages()
        self.export_messages_to_csv()

    def get_random_message(self) -> str:
        return random.choice(self.messages) if self.messages else "¡Feliz Cumpleaños!"

    def send_email(self, person: Person):
        """Envía un correo de cumpleaños a la persona dada."""
        sender_email = "tu_correo@example.com"
        sender_password = "tu_contraseña"

        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = person.email
        message['Subject'] = f"¡Feliz cumpleaños, {person.name}!"
        body = self.get_random_message()
        message.attach(MIMEText(body, 'plain'))

        try:
            with smtplib.SMTP('smtp.gmail.com', 587) as server:
                server.starttls()
                server.login(sender_email, sender_password)
                server.sendmail(sender_email, person.email, message.as_string())
                return f"Correo enviado a {person.name} <{person.email}>"
        except Exception as e:
            return f"Error enviando correo a {person.name}: {e}"

# --- Interfaz Streamlit ---
st.set_page_config(page_title="Recordatorio de Cumpleaños", layout="centered")
st.title("🎉 Recordatorio de Cumpleaños")

manager = BirthdayManager()

menu = st.sidebar.selectbox("Menú", [
    "Registrar Cumpleaños",
    "Ver Próximos Cumpleaños",
    "Añadir Mensaje de Felicitación",
    "Enviar Correos Hoy",
    "Exportar Datos a CSV"
])

if menu == "Registrar Cumpleaños":
    st.header("Registrar Nuevo Cumpleaños")
    name = st.text_input("Nombre")
    birthdate = st.date_input("Fecha de Cumpleaños.")
    email = st.text_input("Correo Electrónico")
    if st.button("Guardar"):
        manager.add_person(name, birthdate.strftime("%Y-%m-%d"), email)
        st.success("Cumpleaños registrado con éxito.")

elif menu == "Ver Próximos Cumpleaños":
    st.header("Cuenta Regresiva para Cumpleaños")
    data = [(p.name, p.birthdate.strftime('%Y-%m-%d'), p.days_until_birthday()) for p in manager.upcoming_birthdays()]
    df = pd.DataFrame(data, columns=["Nombre", "Cumpleaños", "Días restantes"])
    st.dataframe(df)

elif menu == "Añadir Mensaje de Felicitación":
    st.header("Añadir Mensaje de Cumpleaños")
    msg = st.text_area("Mensaje")
    if st.button("Guardar Mensaje"):
        manager.add_message(msg)
        st.success("Mensaje guardado exitosamente.")

elif menu == "Enviar Correos Hoy":
    st.header("Enviar Correos Automáticos")
    enviados = []
    for person in manager.people:
        if person.birthdate.strftime("%m-%d") == datetime.today().strftime("%m-%d"):
            resultado = manager.send_email(person)
            enviados.append(resultado)
    if enviados:
        st.success("\n".join(enviados))
    else:
        st.info("No hay cumpleaños hoy.")

elif menu == "Exportar Datos a CSV":
    st.header("Exportar Datos a CSV")
    manager.export_birthdays_to_csv()
    manager.export_messages_to_csv()
    if os.path.exists(CSV_BIRTHDAYS) and os.path.exists(CSV_MESSAGES):
        st.success("Datos exportados con éxito a archivos CSV.")
        with open(CSV_BIRTHDAYS, 'rb') as f:
            st.download_button("Descargar Cumpleaños", f, file_name=CSV_BIRTHDAYS)
        with open(CSV_MESSAGES, 'rb') as f:
            st.download_button("Descargar Mensajes", f, file_name=CSV_MESSAGES)
    else:
        st.error("Ocurrió un error al exportar los datos.")

ModuleNotFoundError: No module named 'streamlit'